![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [71]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "/content/X_test.txt"
filename_labels = "/content/activity_labels.txt"

filename_subtrain = "/content/subject_train.txt"
filename_xtrain = "/content/X_train.txt"
filename_ytrain = "/content/y_train.txt"

filename_subtest = "/content/subject_test.txt"
ffilename_xtest = "/content/X_test.txt"
filename_ytest = "/content/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None)#, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None)#, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

<ipython-input-71-b6287cc5b559>:21: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
<ipython-input-71-b6287cc5b559>:24: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
<ipython-input-71-b6287cc5b559>:28: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)


In [73]:
# verificando a quantidade de colunas
dfs = [X_train, X_test]

for df in dfs:
  print(df.shape)

(7352, 561)
(2947, 561)


SyntaxError: invalid syntax (<ipython-input-69-1ada7b1aa637>, line 1)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [51]:
# Medindo o tempo de processamento
%%time
# Criação do classificador de árvore de decisão com ccp_alpha=0.001
clf = DecisionTreeClassifier(ccp_alpha=0.001)

# Treinamento do classificador utilizando os dados de treinamento
clf.fit(X_train, y_train)

# Avaliação da acurácia do classificador nos dados de treinamento
print(f'Acurácia na base de treinamento: {clf.score(X_train, y_train)}')
# Avaliação da acurácia do classificador nos dados de teste
print(f'Acurácia na base de teste: {clf.score(X_test, y_test)}\n')

Acurácia na base de treinamento: 0.9757889009793254
Acurácia na base de teste: 0.8802171700033933

CPU times: user 5.65 s, sys: 4.74 ms, total: 5.65 s
Wall time: 6.48 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [52]:
%%time

# Aplica o PCA com 1 componente aos dados de treinamento
prcomp = PCA(n_components=1).fit(X_train)

# Transforma os dados de treinamento e teste utilizando as componentes principais encontradas pelo PCA
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

# Imprime a forma dos dados de treinamento e teste após a transformação
print(f'Dimensões da base de treinamento: {pc_treino.shape}')
print(f'Dimensões da base de teste: {pc_teste.shape}')

# Inicializa um classificador de árvore de decisão com ccp_alpha=0.001 e treina-o com os dados de treinamento transformados
clf = DecisionTreeClassifier(ccp_alpha=0.001)
clf.fit(pc_treino, y_train)

# Calcula e imprime a acurácia do classificador nos dados de treinamento e teste
print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

CPU times: user 627 ms, sys: 296 ms, total: 923 ms
Wall time: 495 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [74]:
%%time

componentes = [1, 2, 5, 10, 50, 100, 200]

# Loop sobre os diferentes números de componentes
for n in componentes:
    # Executa o PCA com o número de componentes atual
    prcomp = PCA(n_components=n).fit(X_train)

    # Transforma os dados de treinamento e teste nos componentes principais
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)

    # Imprime as dimensões dos dados transformados
    print(f'Dimensões da base de treinamento: {pc_treino.shape}')
    print(f'Dimensões da base de teste: {pc_teste.shape}')

    # Cria e treina um classificador de árvore de decisão
    clf = DecisionTreeClassifier(ccp_alpha=0.001)
    clf.fit(pc_treino, y_train)

    # Avalia a acurácia na base de treinamento e teste
    print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
    print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

Dimensões da base de treinamento: (7352, 2)
Dimensões da base de teste: (2947, 2)
Acurácia na base de treinamento: 0.6127584330794341
Acurácia na base de teste: 0.5846623685103495

Dimensões da base de treinamento: (7352, 5)
Dimensões da base de teste: (2947, 5)
Acurácia na base de treinamento: 0.8460282916213275
Acurácia na base de teste: 0.7885985748218527

Dimensões da base de treinamento: (7352, 10)
Dimensões da base de teste: (2947, 10)
Acurácia na base de treinamento: 0.8926822633297062
Acurácia na base de teste: 0.8245673566338649

Dimensões da base de treinamento: (7352, 50)
Dimensões da base de teste: (2947, 50)
Acurácia na base de treinamento: 0.9174374319912949
Acurácia na base de teste: 0.8282999660671869

Dimensões da base de treinamento: (7352, 100)
Dimensões da base de teste: (2947, 100)
Acurá

In [75]:
%%time

# testando novamente com componentes diferentes
componentes = [50, 60, 70, 80]

# Loop sobre os diferentes números de componentes
for n in componentes:
    # Executa o PCA com o número de componentes atual
    prcomp = PCA(n_components=n).fit(X_train)

    # Transforma os dados de treinamento e teste nos componentes principais
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)

    # Imprime as dimensões dos dados transformados
    print(f'Dimensões da base de treinamento: {pc_treino.shape}')
    print(f'Dimensões da base de teste: {pc_teste.shape}')

    # Cria e treina um classificador de árvore de decisão
    clf = DecisionTreeClassifier(ccp_alpha=0.001)
    clf.fit(pc_treino, y_train)

    # Avalia a acurácia na base de treinamento e teste
    print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
    print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 50)
Dimensões da base de teste: (2947, 50)
Acurácia na base de treinamento: 0.919885745375408
Acurácia na base de teste: 0.8282999660671869

Dimensões da base de treinamento: (7352, 60)
Dimensões da base de teste: (2947, 60)
Acurácia na base de treinamento: 0.9212459194776932
Acurácia na base de teste: 0.8245673566338649

Dimensões da base de treinamento: (7352, 70)
Dimensões da base de teste: (2947, 70)
Acurácia na base de treinamento: 0.9182535364526659
Acurácia na base de teste: 0.826942653545979

Dimensões da base de treinamento: (7352, 80)
Dimensões da base de teste: (2947, 80)
Acurácia na base de treinamento: 0.9221980413492927
Acurácia na base de teste: 0.832711231761113

CPU times: user 7.47 s, sys: 2.76 s, total: 10.2 s
Wall time: 7.83 s


In [77]:
%%time

# Executa o PCA com o número de componentes atual
n = 50
prcomp = PCA(n_components=n).fit(X_train)

# Transforma os dados de treinamento e teste nos componentes principais
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

# Imprime as dimensões dos dados transformados
print(f'Dimensões da base de treinamento: {pc_treino.shape}')
print(f'Dimensões da base de teste: {pc_teste.shape}')

# Cria e treina um classificador de árvore de decisão
clf = DecisionTreeClassifier(ccp_alpha=0.001)
clf.fit(pc_treino, y_train)

# Avalia a acurácia na base de treinamento e teste
print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 50)
Dimensões da base de teste: (2947, 50)
Acurácia na base de treinamento: 0.9177094668117519
Acurácia na base de teste: 0.8266033254156769

CPU times: user 1.93 s, sys: 441 ms, total: 2.37 s
Wall time: 4.15 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

A acurária com apenas uma componente ficou abaixo de 50%, porém, quando testamos para mais componentes, verificamos que a acurácia aumentou conforme a quantidade de componentes, obtendo um valor satisfatóriamente próxima à acurácia do modelo com todos os componentes.\
Sobre o tempo de processamento foi possível observar uma melhora significativa, reduzindo o tempo de processamento para menos da metade do tempo inicial.